### 1. 사용할 라이브러리 불러오기

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

### 2. 웹사이트 불러오기

In [19]:
name=['푸르밀 검은콩우유']
category=['별점']

#푸르밀 검은콩우유 후기
ns_address="https://search.shopping.naver.com/catalog/26574850523?query=%ED%91%B8%EB%A5%B4%EB%B0%80%EA%B2%80%EC%9D%80%EC%BD%A9%EC%9A%B0%EC%9C%A0&NaPm=ct%3Dlajdxedc%7Cci%3Dbd179e3cba9e6651be51dbc080049277fe868c33%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D0dbd3220a32f9deab095d316dc4958c6ea7b27cf"
#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul"

### 3. 쇼핑몰 리뷰 불러오기

In [20]:
header = {'User-Agent': ''}
d = webdriver.Chrome('폴더 경로 바꾸기/chromedriver.exe')
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

In [21]:
#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

element=d.find_element(By.XPATH, shoppingmall_review)
d.execute_script("arguments[0].click();", element)
sleep(2)

### 4. 데이터 프레임 만들기

In [24]:
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

### 5. 리뷰 가져오기

In [22]:
# 리뷰 가져오기
d.find_element(By.XPATH, shoppingmall_review).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]

### 6. 리뷰 수집하기

In [23]:
page=1
cnt=1 #리뷰index
while True:
    j=1
    print ("페이지", page ,"\n 출력중입니다..") 
    sleep(2)
    while True: #한 페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
#         if page>=20 and page<=30: 이거하고 밑에부분 tab
        try:
            # 평점
            star=d.find_element(By.XPATH, f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[{j}]/div[1]/span[1]').text
            stars.append(star)
            
            # 리뷰
            if page==1:
                review=d.find_element(By.XPATH, f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[{j}]/div[2]/div[1]/p').text
            else:
                review=d.find_element(By.XPATH, f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[{j}]/div[2]/div/p').text
            reviews.append(review)
            
#             if j%2==0: #화면에 2개씩 보이도록 스크롤
#                 ELEMENT = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]')
#                 d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            print(cnt, review ,star, "\n")
            cnt+=1
            j+=1
        except: break
            
    sleep(2)
    
    if page%10==0: # page가 10, 20, 30, 등 10의 배수 일 때
        if page==10: # 10일 때만 
            next_page=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[11]').click()
        else: # 20, 30, 등 일때만
            next_page=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
    ## page가 10일 때는 다음 버튼이 11이고 20, 30, 등은 다음 버튼이 12

    else:
        if page<=10:
                next_page=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+1)+']').click() 
        else:
            next_page=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
    ## page가 11부터는 이전(1), 1(2), 2(3), ... 10(11), 다음(12)로 되어 있음

    page+=1

페이지 1 
 출력중입니다..
1 푸르밀 검은콩 우유 정말 맛있어요 저희 가족들이 정말 좋아하는 우유에요! 어렸을때 우연히 푸르밀 검은콩우유를 처음 먹게 된 이후 그맛에 빠져 아직까지도 애용하고있는 제품입니다!! 너무 맛있어서 꿀떡꿀떡 먹는데 ㅋ 맨날 오빠랑 서로 먹겠다고 싸움이 날정도로 정말 좋아하고 가족들도 정말 좋아하는 제품입니다. 검은콩 우유를 가장 좋아하는 이유는..음..왜냐면 맛있기 때문이에요! 무엇보다 고소해서 맛있어요!처음 사서 먹을려고 했을때 별로지않을까..맛이 없지 않을까 하면서 속는셈치고 먹어봤는디 ㅋㅋ 맛있고 그 특유의 고소함과 꾸덕함에 빠져서 지금까지헤어나오지를 못하고 있는 중입니다!!😀😀 점도라고 해야할까요? 우유 자체가 약간꾸덕한 느낌!! 약간 생크림 들어간것처럼 부드럽고 막 그런느낌이 있어요 !!!!💝💝최고 그리고!! 재료자체가 믿을만해서 좋아요 국내산 검은콩 약콩쥐눈이콩 진짜 성분부터가 믿을 만한 느낌!!! 국내산이라니!! !!꺄아아!!진짜 사랑하는 푸르밀검은콩우유 이젠 마트가 아니더라도 이렇게 푸르밀 공식스토어에서 배송도 편하고 쉽게 먹을수있다는게 기뻐요 지금 산지 하루되었는데 벌써 두통다 먹었어요 ㅋㅋ 아 ...진짜 사랑합니다 !! 공식스토어에서 자주자주 사먹을꺼에요!! 이맛 놓칠수없어!!!✌✌ 푸르밀 검은콩우유는 찐사랑♥ 공식스토어에서 구매해서 냠냠냠먹기 찐행복이죵 평점5 

2 어렸을 때 제가 콩을 싫어해서 부모님이 매일 이걸 사주셨어요. 너무 맛있어서 매일 이거 한통씩 먹었습니다. 성인이 되서도 가끔 생각나서 하나씩 사먹긴 했었는데 갑자기 푸르밀이 판매중단한다고해서 급하게 시켜봤습니다.
아직도 맛있는데....정말 아쉬워요.. 평점5 

3 유통기한이 짧아서 아쉽지만 이거만큼 맛있는 검은콩 우유는 못봤습니다. 이제 못본다는게 아쉽지만 푸르밀 직원만큼은 아니겠죠... 그동안 고생 많으셨습니다
30일까지 잘 마무리 하시고 다른 곳에서는 좋은 날만 있기를 기도합니다 평점5 

4 주문하고 송장도 금방 뜨고 발송했다길래
계속 기다

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[10]"}
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x00471ED3+2236115]
	Ordinal0 [0x004092F1+1807089]
	Ordinal0 [0x003166FD+812797]
	Ordinal0 [0x003455DF+1005023]
	Ordinal0 [0x003457CB+1005515]
	Ordinal0 [0x00377632+1209906]
	Ordinal0 [0x00361AD4+1120980]
	Ordinal0 [0x003759E2+1202658]
	Ordinal0 [0x003618A6+1120422]
	Ordinal0 [0x0033A73D+960317]
	Ordinal0 [0x0033B71F+964383]
	GetHandleVerifier [0x0071E7E2+2743074]
	GetHandleVerifier [0x007108D4+2685972]
	GetHandleVerifier [0x00502BAA+532202]
	GetHandleVerifier [0x00501990+527568]
	Ordinal0 [0x0041080C+1837068]
	Ordinal0 [0x00414CD8+1854680]
	Ordinal0 [0x00414DC5+1854917]
	Ordinal0 [0x0041ED64+1895780]
	BaseThreadInitThunk [0x75ADFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77B97BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77B97B8E+238]


In [30]:
df4=add_dataframe(name_,category_,reviews,stars,cnt)

In [31]:
df4

,type,category,review,star
1,푸르밀 검은콩우유,별점,푸르밀 검은콩 우유 정말 맛있어요 저희 가족들이 정말 좋아하는 우유에요! 어렸을때 ...,평점5
2,푸르밀 검은콩우유,별점,어렸을 때 제가 콩을 싫어해서 부모님이 매일 이걸 사주셨어요. 너무 맛있어서 매일 ...,평점5
3,푸르밀 검은콩우유,별점,유통기한이 짧아서 아쉽지만 이거만큼 맛있는 검은콩 우유는 못봤습니다. 이제 못본다는...,평점5
4,푸르밀 검은콩우유,별점,주문하고 송장도 금방 뜨고 발송했다길래\n계속 기다렸었는데 3일이 되도록 안왔어요\...,평점5
5,푸르밀 검은콩우유,별점,1월21일까지네요 유통기한이....\n푸르밀 검은콩 우유 자주 먹는데 여기선 처음 ...,평점4
...,...,...,...,...
744,푸르밀 검은콩우유,별점,맛나요맛나요맛나요맛나요,평점5
745,푸르밀 검은콩우유,별점,굿~~<<<<~~~~~~~~~~~~,평점5
746,푸르밀 검은콩우유,별점,Okokokokokok,평점5
747,푸르밀 검은콩우유,별점,굿~~~~~~~~~~,평점5
